In [3]:
!pip3 install --upgrade azureml-sdk[notebooks,automl]

# install experiment widget
!jupyter nbextension install --py --user azureml.train.widgets

# enable experiment widget
!jupyter nbextension enable --py --user azureml.train.widgets

The directory '/home/chenhui/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/chenhui/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |████████████████████████████████| 788kB 1.9MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 10.3MB/s ta 0:00:01
    100% |████████████████████████████████| 8.2MB 215kB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 14.9MB/s ta 0:00:01
    100% |████████████████████████████████| 706kB 2.3MB/s eta 0:00:01
    100% |████████████████████████████████| 624kB 2.4MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 13.8MB/s ta 0:00:01
    100% |██████

    100% |████████████████████████████████| 706kB 1.8MB/s ta 0:00:011
    100% |████████████████████████████████| 143kB 11.0MB/s ta 0:00:01
Requirement already up-to-date: six>=1.11.0 in /usr/local/lib/python3.5/dist-packages (from azureml-core==0.1.74.*->azureml-sdk[automl,notebooks])
    100% |████████████████████████████████| 112kB 13.6MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 13.2MB/s ta 0:00:01
    100% |████████████████████████████████| 471kB 2.9MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 13.2MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 11.3MB/s ta 0:00:01
    100% |████████████████████████████████| 512kB 2.4MB/s ta 0:00:011
    100% |████████████████████████████████| 112kB 13.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.1MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 14.7MB 118kB/s eta 0:00:01
Requirement already up-to-date: scipy<=1.1.0,>=1.0.0 in /home/chenhui/.local/lib/py

    100% |████████████████████████████████| 81kB 14.2MB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 10.5MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 14.4MB/s ta 0:00:01
    100% |████████████████████████████████| 204kB 5.2MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 13.5MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 11.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 13.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 112kB 12.4MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 12.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 14.7MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 4.3MB/s eta 0:00:01
Requirement already up-to-date: pyyaml~=3.13 in /home/chenhui/.local/lib/python3.5/site-packages (from azure-cli-core>=2.0.38->azureml-core==0.1.74.*->azureml-sdk[aut

    100% |████████████████████████████████| 430kB 2.3MB/s eta 0:00:01
    100% |████████████████████████████████| 28.7MB 57kB/s  eta 0:00:01
    100% |████████████████████████████████| 1.3MB 1.2MB/s eta 0:00:01
Requirement already up-to-date: absl-py>=0.1.6 in /usr/local/lib/python3.5/dist-packages (from tensorflow==1.10.0->azureml-train-automl==0.1.74.*; extra == "automl"->azureml-sdk[automl,notebooks])
Requirement already up-to-date: setuptools<=39.1.0 in /usr/local/lib/python3.5/dist-packages (from tensorflow==1.10.0->azureml-train-automl==0.1.74.*; extra == "automl"->azureml-sdk[automl,notebooks])
    100% |████████████████████████████████| 3.3MB 551kB/s eta 0:00:01
    100% |████████████████████████████████| 9.7MB 179kB/s eta 0:00:01
Requirement already up-to-date: protobuf>=3.6.0 in /usr/local/lib/python3.5/dist-packages (from tensorflow==1.10.0->azureml-train-automl==0.1.74.*; extra == "automl"->azureml-sdk[automl,notebooks])
Requirement already up-to-date: gast>=0.2.0 in /usr/l

Requirement already up-to-date: backcall in /usr/local/lib/python3.5/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->azureml-train-widgets==0.1.74.*; extra == "notebooks"->azureml-sdk[automl,notebooks])
    100% |████████████████████████████████| 348kB 3.7MB/s eta 0:00:01
Requirement already up-to-date: pexpect; sys_platform != "win32" in /usr/local/lib/python3.5/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->azureml-train-widgets==0.1.74.*; extra == "notebooks"->azureml-sdk[automl,notebooks])
Requirement already up-to-date: pickleshare in /usr/local/lib/python3.5/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->azureml-train-widgets==0.1.74.*; extra == "notebooks"->azureml-sdk[automl,notebooks])
Requirement already up-to-date: decorator in /usr/local/lib/python3.5/dist-packages (from ipython>=4.0.0; python_version >= "3.3"->ipywidgets>=7.0.0->azureml-train-widgets==0.1.74.*; extra == "noteb

    Uninstalling python-dateutil-2.7.3:
      Successfully uninstalled python-dateutil-2.7.3
  Found existing installation: urllib3 1.13.1
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, outside environment /usr
  Found existing installation: oauthlib 1.0.3
    Not uninstalling oauthlib at /usr/lib/python3/dist-packages, outside environment /usr
  Found existing installation: idna 2.0
    Not uninstalling idna at /usr/lib/python3/dist-packages, outside environment /usr
  Found existing installation: chardet 2.3.0
    Not uninstalling chardet at /usr/lib/python3/dist-packages, outside environment /usr
  Found existing installation: requests 2.9.1
    Not uninstalling requests at /usr/lib/python3/dist-packages, outside environment /usr
  Found existing installation: cryptography 1.2.3
    Not uninstalling cryptography at /usr/lib/python3/dist-packages, outside environment /usr
  Running setup.py install for cryptography ... done
  Found existing installation: PyJWT 1.3.0


In [4]:
%matplotlib inline
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt

In [5]:
import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.74


In [6]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

In [7]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /home/chenhui/TSPerf/retail_sales/OrangeJuice_Pt_3Weeks_Weekly/submissions/DilatedCNN/config.json
Workspace name: chhws
Azure region: southcentralus
Subscription id: ff18d7a8-962a-406c-858f-49acd23d6c01
Resource group: tsperf


## Create an Azure ML experiment

In [11]:
from azureml.core import Experiment

#script_folder = './tf-mnist'
#os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='tune_dcnn')

## Validate Script Locally

In [20]:
from azureml.core.runconfig import RunConfiguration
run_config_user_managed = RunConfiguration()
run_config_user_managed.environment.python.user_managed_dependencies = True
run_config_user_managed.environment.python.interpreter_path = '/usr/bin/python3.5'

In [21]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./', 
                      script='train_score_all.py', 
                      arguments=['--data-folder', '/home/chenhui/TSPerf/retail_sales/OrangeJuice_Pt_3Weeks_Weekly/data/', '--dropout-rate', '0.3'],
                      run_config=run_config_user_managed)
run_local = exp.submit(src)

In [23]:
run_local.fail

<bound method Run.fail of Run(Experiment: tune_dcnn,
Id: tune_dcnn_1542658241_5c84c784,
Type: azureml.scriptrun,
Status: Completed)>

In [24]:
run_local.get_details()
run_local.get_metrics()

{'MAPE': 52.905323745857956}

## Upload Orange Juice dataset to default datastore

In [9]:
ds = ws.get_default_datastore()
ds.upload(src_dir='../../data', target_path='data', overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_1094968f03694512b46340b93f7d2c59

## Create Batch AI cluster as compute target

In [28]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    # look for the existing cluster by name
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    if type(compute_target) is BatchAiCompute:
        print('Found existing compute target {}.'.format(cluster_name))
    else:
        print('{} exists but it is not a Batch AI cluster. Please choose a different name.'.format(cluster_name))
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size="STANDARD_NC6", # GPU-based VM
                                                                #vm_priority='lowpriority', # optional
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize())

Found existing compute target gpucluster.


In [12]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

gpucluster BatchAI Succeeded


## Configure Docker Environment

In [25]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'numpy', 'scipy', 'scikit-learn', 'tensorflow-gpu', 'keras', 'joblib'],
                                                         python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

## Create Estimator

In [38]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureFile chhws9475974549 azureml-filestore-87c07697-2faa-418a-a879-bbaddc5c7cac


In [39]:
ds.upload(src_dir='../../data', target_path='./data')

Target already exists. Skipping upload for ./data/yx.csv
Target already exists. Skipping upload for ./data/.gitignore
Target already exists. Skipping upload for ./data/storedemo.csv
Target already exists. Skipping upload for ./data/test/test_round_1.csv
Target already exists. Skipping upload for ./data/test/test_round_10.csv
Target already exists. Skipping upload for ./data/test/test_round_2.csv
Target already exists. Skipping upload for ./data/test/test_round_12.csv
Target already exists. Skipping upload for ./data/test/test_round_9.csv
Target already exists. Skipping upload for ./data/test/test_round_6.csv
Target already exists. Skipping upload for ./data/test/test_round_4.csv
Target already exists. Skipping upload for ./data/test/test_round_3.csv
Target already exists. Skipping upload for ./data/test/test_round_8.csv
Target already exists. Skipping upload for ./data/test/test_round_5.csv
Target already exists. Skipping upload for ./data/test/test_round_7.csv
Target already exists. S

$AZUREML_DATAREFERENCE_509d5ab95faf4f2c9caf6db5ae981340

path_on_datastore = 'data'
ds_data = ds.path(path_on_datastore)
print(ds_data)

In [44]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--data-folder': ds_data.as_mount(),
    '--dropout-rate': 0.3,
    '--learning-rate': 0.01
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='train_validate.py',
                environment_definition=env)

In [45]:
# Submit job
run_batchai = exp.submit(config=est)

In [46]:
# Check job status
from azureml.train.widgets import RunDetails
RunDetails(run_batchai).show()

_UserRun(widget_settings={'childWidgetDisplay': 'popup'})

In [36]:
run_batchai.get_details()
run_batchai.get_metrics()

{}

## Copy the training files into the script folder

In [14]:
from azureml.train.dnn import TensorFlow

In [ ]:
script_folder = './tune-dcnn'
os.makedirs(script_folder, exist_ok=True)